In [142]:
%matplotlib inline
import matplotlib.pyplot as plt
import pyreadr
import pandas as pd
import numpy as np

In [143]:
# https://gist.github.com/LeiG/8094753a6cc7907c716f?permalink_comment_id=2795790#gistcomment-2795790
fl2 = pyreadr.read_r('../data/Nurhayu2020/full_pedigree_for_Table_1.RData')['fl2']
print(fl2.shape)
fl2.head()

(11933, 6)


,famille,animal,sex,Idfather,Idmother,handedness
rownames,,,,,,
2,1_bajawa,1_bajawamother2,F,NaN,NaN,0.0
3,1_bajawa,1_bajawafather,M,NaN,NaN,0.0
4,1_bajawa,1_bajawamother,F,NaN,NaN,0.0
5,1_bajawa,1_bajawas1,M,1_bajawafather,1_bajawamother,0.0
6,1_bajawa,1_bajawas2,M,1_bajawafather,1_bajawamother,0.0


In [144]:
Allego = pyreadr.read_r('../data/Nurhayu2020/ego_individuals.RData')['Allego']
print(Allego.shape)

(700, 6)


In [145]:
mock_individuals = pyreadr.read_r('../data/Nurhayu2020/mock_individuals.RData')['mock_individuals']['mock_individuals']
print(mock_individuals.shape)
mock_individuals.head()

(180,)


0    115_wangkafatherwife
1    115_wangkamotherwife
2              104_wruto1
3              104_wrutO2
4              204_nruti1
Name: mock_individuals, dtype: object

In [146]:
print("Ego:")
print("n={}, f={:.3f}".format(Allego.shape[0], Allego['handedness'].dropna().mean()))

Ego:
n=700, f=0.146


In [147]:
fami = fl2['famille'].unique()

IDEgofather = []
IDEgomother = []
for i in range(len(fami)):
    fami_idx = fl2['famille'] == fami[i]
#     print('fami', fami[i])
    ped = fl2[fami_idx]
    ped_idx = ped['animal'].isin(Allego['animal'])
    IDego = ped.loc[ped_idx, 'animal']
#     print('ego', IDego)
    localfather = ped.loc[ped['animal'].isin(IDego), 'Idfather']
    localmother = ped.loc[ped['animal'].isin(IDego), 'Idmother']
#     print('father', localfather)
#     print('mother', localmother)
    IDEgofather.extend(localfather.values)
    IDEgomother.extend(localmother.values)

In [148]:
father = set(IDEgofather)
print(len(father))
mother = set(IDEgomother)
print(len(mother))

683
683


In [149]:
fathers = fl2[fl2['animal'].isin(father)]
mothers = fl2[fl2['animal'].isin(mother)]

fathers = fathers[~fathers['animal'].isin(mock_individuals)]
mothers = mothers[~mothers['animal'].isin(mock_individuals)]

parents = pd.concat([fathers, mothers])
mother_father = parents['animal']

In [150]:
parents = parents[~np.isnan(parents['handedness'])]

In [151]:
print("Parents:")
print("n={}, f={:.3f}".format(parents.shape[0], parents['handedness'].dropna().mean()))

Parents:
n=1344, f=0.063


In Table 1, there are 1336 parents, because parents that are also children were removed.

# Laland 1995-style data

In [154]:
Allego.head()

,famille,animal,sex,Idfather,Idmother,handedness
rownames,,,,,,
8,1_bajawa,1_bajawaego,F,1_bajawafather,1_bajawamother,0.0
25,1_bajawa,1_bajawaego2,F,1_bajawahusband,1_bajawaego,0.0
44,3_bajawa,3_bajawaego,F,3_bajawafather,3_bajawamother,0.0
59,7_langa,7_langaego,M,7_langafather,7_langamother,1.0
75,8_langa,8_langaego,M,8_langafather,8_langamother,0.0


In [153]:
parents.head()

,famille,animal,sex,Idfather,Idmother,handedness
rownames,,,,,,
3,1_bajawa,1_bajawafather,M,NaN,NaN,0.0
23,1_bajawa,1_bajawahusband,M,NaN,NaN,0.0
39,3_bajawa,3_bajawafather,M,NaN,NaN,0.0
57,7_langa,7_langafather,M,7_langaGP2,7_langaGP1,1.0
72,8_langa,8_langafather,M,NaN,NaN,0.0


In [170]:
child_father = pd.merge(Allego, parents[['animal', 'handedness']], left_on='Idfather', right_on='animal', suffixes=('_child', '_father'))
child_father.head()

,famille,animal_child,sex,Idfather,Idmother,handedness_child,animal_father,handedness_father
0,1_bajawa,1_bajawaego,F,1_bajawafather,1_bajawamother,0.0,1_bajawafather,0.0
1,1_bajawa,1_bajawaego2,F,1_bajawahusband,1_bajawaego,0.0,1_bajawahusband,0.0
2,3_bajawa,3_bajawaego,F,3_bajawafather,3_bajawamother,0.0,3_bajawafather,0.0
3,7_langa,7_langaego,M,7_langafather,7_langamother,1.0,7_langafather,1.0
4,8_langa,8_langaego,M,8_langafather,8_langamother,0.0,8_langafather,0.0


In [169]:
child_mother = pd.merge(Allego, parents[['animal', 'handedness']], left_on='Idmother', right_on='animal', suffixes=('_child', '_mother'))
child_mother.head()

,famille,animal_child,sex,Idfather,Idmother,handedness_child,animal_mother,handedness_mother
0,1_bajawa,1_bajawaego,F,1_bajawafather,1_bajawamother,0.0,1_bajawamother,0.0
1,1_bajawa,1_bajawaego2,F,1_bajawahusband,1_bajawaego,0.0,1_bajawaego,0.0
2,3_bajawa,3_bajawaego,F,3_bajawafather,3_bajawamother,0.0,3_bajawamother,0.0
3,7_langa,7_langaego,M,7_langafather,7_langamother,1.0,7_langamother,0.0
4,8_langa,8_langaego,M,8_langafather,8_langamother,0.0,8_langamother,0.0


In [179]:
child_parent = pd.merge(
    child_mother[['animal_child', 'handedness_child', 'handedness_mother']],
    child_father[['animal_child', 'handedness_father']],
    on='animal_child'
)
child_parent.set_index('animal_child', inplace=True)
print(child_parent.shape)
child_parent.head()

(683, 3)


,handedness_child,handedness_mother,handedness_father
animal_child,,,
1_bajawaego,0.0,0.0,0.0
1_bajawaego2,0.0,0.0,0.0
3_bajawaego,0.0,0.0,0.0
7_langaego,1.0,0.0,1.0
8_langaego,0.0,0.0,0.0


In [180]:
R_mother = child_parent['handedness_mother'] == 0
R_father = child_parent['handedness_father'] == 0
R_child = child_parent['handedness_child'] == 0

In [184]:
R_RR = (R_mother & R_father & R_child).sum()
L_RR = (R_mother & R_father & ~R_child).sum()
R_RL = (~R_mother & R_father & R_child).sum() + (R_mother & ~R_father & R_child).sum()
L_RL = (~R_mother & R_father & ~R_child).sum() + (R_mother & ~R_father & ~R_child).sum()
R_LL = (~R_mother & ~R_father & R_child).sum()
L_LL = (~R_mother & ~R_father & ~R_child).sum()

In [186]:
assert R_RR + L_RR + R_RL + L_RL + R_LL + L_LL == child_parent.shape[0]

In [187]:
R_RR, L_RR , R_RL , L_RL , R_LL , L_LL

(521, 73, 61, 26, 1, 1)